# Introducción a SQL y Conexión con Python (sqlite3)

**Curso:** Fundamentos de Programación y Analítica de Datos con Python  
**Duración estimada del bloque:** 2 horas  
**Autores:** Jhon Erik Navarrete Gómez
**Creado:** 2025-09-16T18:16:55Z

## Objetivos específicos
- Conectar a una base de datos SQLite desde Python utilizando el módulo estándar `sqlite3`.
- Crear tablas y cargar datos mínimos de ejemplo aplicando tipos apropiados.
- Ejecutar consultas básicas: `SELECT`, filtrado con `WHERE` y combinaciones con `JOIN`.
- Transportar resultados de SQL hacia `pandas` para su inspección y posterior análisis.
- Adoptar buenas prácticas de manejo de conexiones, cursores y parametrización de consultas.

## Prerrequisitos
- Conocimientos básicos de Python (variables, tipos, funciones) y uso de `pip`/entornos virtuales.
- Nociones elementales de archivos y rutas.


## Tema 1. Fundamentos de SQLite y conexión con `sqlite3`

### Definición
SQLite es un motor de base de datos relacional ligero que almacena toda la información en un único archivo local. En Python, el módulo estándar `sqlite3` provee la interfaz para crear bases de datos, ejecutar instrucciones SQL y obtener resultados mediante conexiones y cursores.

### Importancia en programación y analítica de datos
- Permite persistir datos estructurados de forma relacional sin necesidad de desplegar un servidor.
- Facilita prototipos rápidos y reproductibles para análisis locales y notebooks.
- Ofrece compatibilidad con SQL estándar suficiente para la mayoría de tareas analíticas iniciales.

### Buenas prácticas profesionales y errores comunes
- **Parametrizar consultas** para evitar inyección SQL y mejorar legibilidad: usar `?` y tuplas de parámetros.
- **Gestionar recursos**: cerrar cursores y conexiones o usar context managers (`with`).
- **Commit explícito** tras operaciones de escritura (`INSERT`, `UPDATE`, `DELETE`).
- **Tipos y nulabilidad**: definir columnas con tipos y restricciones coherentes (por ejemplo, `NOT NULL`).

### Ejemplo en Python: conexión, creación de tablas e inserciones
El siguiente ejemplo usa una base en memoria (`:memory:`) para fines demostrativos.


In [3]:

# TODO: Conexión y creación de tablas básicas con sqlite3
import sqlite3

# Sugerencia: usar ':memory:' para ejemplos temporales; usar 'empresa.db' para persistencia en archivo.
conn = sqlite3.connect(':memory:')  # para persistir a disco, usar 'empresa.db'
cur = conn.cursor()

# Crear tabla empleados
cur.execute('''
CREATE TABLE empleados (
    id INTEGER PRIMARY KEY,
    nombre TEXT NOT NULL,
    departamento TEXT NOT NULL,
    salario REAL NOT NULL CHECK (salario >= 0)
);
''')

empleados = [
    (1, "Ana",   "Ventas", 3000.0),
    (2, "Luis",  "TI",     4200.0),
    (3, "María", "RRHH",   3500.0),
    (4, "Jorge", "TI",     3800.5),
]

cur.executemany('INSERT INTO empleados (id, nombre, departamento, salario) VALUES (?, ?, ?, ?);', empleados)
conn.commit()

# Selección Simple
cur.execute('SELECT * FROM empleados;')
rows = cur.fetchall()
print(rows)


[(1, 'Ana', 'Ventas', 3000.0), (2, 'Luis', 'TI', 4200.0), (3, 'María', 'RRHH', 3500.0), (4, 'Jorge', 'TI', 3800.5)]


## Tema 2. Consultas básicas: `SELECT` y filtrado con `WHERE`

### Definición
`SELECT` recupera columnas desde una o más tablas; `WHERE` limita los registros según condiciones lógicas (comparaciones, `AND`, `OR`, `LIKE`, etc.).

### Importancia en programación y analítica de datos
- Permite construir vistas parciales del dataset para responder preguntas específicas.
- Reduce el volumen de datos a cargar en memoria.
- Es el primer paso antes de agregar, unir y modelar información.

### Buenas prácticas profesionales y errores comunes
- Seleccionar solo columnas necesarias para mantener consultas claras y eficientes.
- Evitar filtros ambiguos: usar comparaciones explícitas y paréntesis al combinar condiciones.
- Verificar unidades y dominios de las columnas antes de filtrar (por ejemplo, salarios netos vs brutos).

### Ejemplo en Python


In [4]:

# TODO: SELECT específico y filtrado con WHERE
cur.execute('SELECT nombre, salario FROM empleados WHERE departamento = ?;', ('TI',))
rows = cur.fetchall()
print(rows)

cur.execute('SELECT nombre, salario FROM empleados ORDER BY salario DESC;')
rows = cur.fetchall()
print(rows)


[('Luis', 4200.0), ('Jorge', 3800.5)]
[('Luis', 4200.0), ('Jorge', 3800.5), ('María', 3500.0), ('Ana', 3000.0)]


In [5]:

# TODO: Filtro con WHERE: salarios mayores a 3,500
min_salario = 3500
cur.execute("SELECT nombre, departamento, salario FROM empleados WHERE salario > ? ORDER BY salario DESC;",(min_salario,))
rows = cur.fetchall()
print(rows)

[('Luis', 'TI', 4200.0), ('Jorge', 'TI', 3800.5)]


In [8]:

# TODO: Expresiones en SELECT y WHERE
# Empeados de TI con un aumento del 5%
departamento_favorecido = 'TI'
cur.execute('''
  SELECT nombre, departamento, salario, ROUND(salario*1.05, 2) AS salario_ajustado
  FROM empleados
  WHERE departamento = ?;
''', (departamento_favorecido,))
rows = cur.fetchall()
print(rows)

[('Luis', 'TI', 4200.0, 4410.0), ('Jorge', 'TI', 3800.5, 3990.53)]


## Tema 3. Combinación de tablas con `JOIN`

### Definición
`JOIN` combina filas de dos tablas según una condición de igualdad entre claves relacionadas (por ejemplo, clave foránea ↔ clave primaria).

### Importancia en programación y analítica de datos
- Integra múltiples fuentes/tablas para responder preguntas de negocio.
- Evita duplicidad y promueve normalización, facilitando el mantenimiento y la calidad de datos.

### Buenas prácticas profesionales y errores comunes
- Establecer claves coherentes (por ejemplo, `id_departamento` numérico) para uniones robustas.
- Nombrar alias de tablas (`e`, `d`) para consultas legibles.
- Verificar cardinalidades (1:1, 1:N, N:M) para evitar duplicación involuntaria de registros.

### Ejemplo en Python


In [ ]:

# TODO: Crear tabla departamentos y poblarla
cur.execute('''
CREATE TABLE departamentos (
  id INTEGER PRIMARY KEY,
  nombre TEXT NOT NULL,
  jefe TEXT NOT NULL
);
''')

departamentos = [
  (1, "Ventas", "Ana"),
  (2, "TI",     "Luis"),
  (3, "RRHH",   "María"),
]

cur.executemany("INSERT INTO departamentos (id, nombre, jefe) VALUES (?, ?, ?);", departamentos)
conn.commit()

# JOIN entre empleados y departamentos
cur.execute('''SELECT
  e.nombre AS empleado,
  e.departamento,
  e.salario,
  d.jefe AS jefe_departamento
  FROM empleados e
  JOIN departamentos d ON e.departamento = d.nombre
  ORDER BY e.id;
''')
cur.fetchall()

[('Ana', 'Ventas', 3000.0, 'Ana'),
 ('Luis', 'TI', 4200.0, 'Luis'),
 ('María', 'RRHH', 3500.0, 'María'),
 ('Jorge', 'TI', 3800.5, 'Luis')]

## Tema 4. Consulta hacia `pandas` para inspección y análisis

### Definición
`pandas` puede ejecutar sentencias `SELECT` sobre una conexión SQLite y retornar un `DataFrame`, facilitando análisis tabulares y visualización posterior.

### Importancia en programación y analítica de datos
- Acelera el flujo exploratorio: de consulta SQL a análisis/visualización sin fricción.
- Permite integrar transformaciones relacionales (SQL) con operaciones vectorizadas/tabulares (pandas).

### Buenas prácticas profesionales y errores comunes
- Limitar el alcance de la consulta SQL al subconjunto relevante y luego continuar en `pandas`.
- Mantener consistencia de tipos (por ejemplo, convertir fechas/categorías) tras la carga.
- Evitar cargar tablas enormes sin filtros (aplicar `WHERE`/`LIMIT` cuando sea posible).

### Ejemplo en Python


In [ ]:

# TODO: Llevar resultados a pandas

## Ejercicios integradores

A continuación, se proponen ejercicios que integran los temas del bloque. Cada ejercicio incluye contexto, datos de entrada, requerimientos, criterios de aceptación y pistas. Tras cada enunciado se presenta una **solución orientativa**.


### Ejercicio 1. Nómina mensual por departamento

**Contexto técnico**  
Eres analista en una empresa mediana. Debes construir un reporte de nómina para Recursos Humanos que resuma el salario total y promedio por departamento para verificar el presupuesto mensual.

**Datos/entradas**  
Usa las tablas `empleados` y `departamentos` ya creadas en este notebook (o recrea una base equivalente).

**Requerimientos**  
1. Escribir una consulta SQL que calcule salario **total** y **promedio** por departamento.  
2. Llevar los resultados a `pandas` y ordenar de mayor a menor por salario total.  
3. Redondear los promedios a dos decimales.

**Criterios de aceptación**  
- El resultado contiene las columnas: `departamento`, `salario_total`, `salario_promedio`.  
- Orden descendente por `salario_total`.  
- Promedios redondeados a 2 decimales.

**Pistas**  
- Usar `GROUP BY departamento`.  
- Para el redondeo en `pandas`, usar `.round(2)`.


In [ ]:

# TODO: Solución Ejercicio 1

### Ejercicio 2. Empleados por encima de umbral

**Contexto técnico**  
El CFO requiere un listado de empleados que superan un umbral de salario para revisar bandas salariales.

**Datos/entradas**  
Umbral configurable (por ejemplo, 3600.0).

**Requerimientos**  
1. Parametrizar la consulta con `?` y pasar el umbral desde Python.  
2. Devolver `nombre`, `departamento`, `salario` ordenados por `salario` descendente.

**Criterios de aceptación**  
- El umbral no está *hardcodeado* en la cadena SQL.  
- El resultado está correctamente ordenado y muestra solo las columnas solicitadas.

**Pistas**  
- Preparar la tupla de parámetros `(umbral,)`.


In [ ]:

# TODO: Solución Ejercicio 2

### Ejercicio 3. Reporte con JOIN: jefe por empleado

**Contexto técnico**  
RRHH necesita saber quién es el jefe de cada empleado para enviar comunicaciones formales por área.

**Datos/entradas**  
Tablas `empleados` y `departamentos` (relación por nombre de departamento).

**Requerimientos**  
1. Realizar un `JOIN` que devuelva `empleado`, `departamento`, `jefe`.  
2. Exportar los resultados a un `DataFrame` y verificar duplicados.

**Criterios de aceptación**  
- No hay filas duplicadas inesperadas.  
- Las columnas tienen nombres claros (`empleado`, `departamento`, `jefe`).

**Pistas**  
- Usar alias de tablas y renombrar columnas con `AS`.


In [ ]:

# TODO: Solución Ejercicio 3

### Ejercicio 4. Ventas por empleado (modelo mínimo)

**Contexto técnico**  
El área comercial pide un tablero sencillo con ventas por empleado para evaluar desempeño. Crearás un modelo mínimo con una tabla `ventas` y harás un informe.

**Datos/entradas**  
Crear tabla `ventas(id INTEGER PRIMARY KEY, id_empleado INTEGER, monto REAL NOT NULL)` y cargar al menos 6 registros que distribuyan ventas entre distintos empleados.

**Requerimientos**  
1. Crear la tabla `ventas` y poblarla.  
2. Construir una consulta que devuelva ventas **totales** por empleado (nombre) usando `JOIN` contra `empleados`.  
3. Cargar el resultado a `pandas` y ordenar descendentemente por ventas totales.

**Criterios de aceptación**  
- Columnas finales: `empleado`, `ventas_totales`.  
- Orden descendente y montos no negativos.  
- Al menos 6 ventas registradas.

**Pistas**  
- `JOIN` por `e.id = v.id_empleado`.  
- Usar `GROUP BY e.id, e.nombre`.


In [ ]:

# TODO: Solución Ejercicio 4